Manipolazione dati e popolamento delle tabelle Utente, Birra, Consumazione

In [1]:
# Funzioni e imports
import json, os
import mysql.connector

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")


def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [2]:
# Connessione al DB
db = mysql.connector.connect(host="localhost", user="arzo", password="abcd1234", database="beer")
cursor = db.cursor()

In [ ]:
jsonUtenti = read_json("utente.json")

#Dati casuali generati con mockaroo.com, ma l'attributo 'conto' era in forma "123,45", mentre il DB necessita di 123.45, quindi converto
for user in jsonUtenti:
    user["saldo"] = float(user["saldo"].replace(",", "."))
    print(user["saldo"])

# Magari serve più tardi...
serialize_json("./", "utente.json", jsonUtenti) 

# Popolamento Utente
for utente in jsonUtenti:
    cursor.execute(
        f'INSERT INTO Utente(id,nome, cognome, email, psw, saldo, data_reg) VALUES (\'{utente["id"]}\',\'{utente["nome"]}\',\'{utente["cognome"]}\',\'{utente["email"]}\',\'{utente["psw"]}\',{utente["saldo"]},\'{utente["data_reg"]}\')'
    )

In [7]:
jsonBirre = read_json("birra.json")
cursor.execute('DELETE FROM Birra')
for birra in jsonBirre:
    cursor.execute(f'INSERT INTO Birra(id, nome, prezzo_litro, disp, gradi, tipo) VALUES ({birra["id"]},\'{birra["nome"]}\',{birra["prezzo_litro"]},\'{birra["disp"]}\',{birra["gradi"]},\'{birra["tipo"]}\')')
    print(f'Inserita birra {birra["nome"]}')

Data read from path: birra.json
Inserita birra Crux
Inserita birra Isaac
Inserita birra Black Hole
Inserita birra Lone Wolf
Inserita birra Pop
Inserita birra Heineken
Inserita birra Guinness
Inserita birra Guerrilla
Inserita birra Kilkenny RED
Inserita birra Peroni
Inserita birra Grim Reaper
Inserita birra Poretti IPA
